# Tipos

Un tipo es básicamente qué clase de "ente" es un valor dado, ya hemos hablado de algunos de estos, tales son: flotantes(Float), enteros(Int), cadenas de caracteres(Strings), funciones, etc.

Podemos declar el tipo de una variable mediante la sintáxis `var::Tipo = valor`(por ahora Julia sólo permite declarar el tipo de una variable dentro de funciones) esto es algo muy parecido en lo que se hace en algunos lenguajes como C.

In [1]:
function test()
    a::Float64 = 2.
    a
end

test (generic function with 1 method)

También podemos asegurarnos de que una expresión dada devulve el tipo correcto 

In [2]:
(1 + 2)::Float64

TypeError: TypeError: in typeassert, expected Float64, got Int64

En Julia existe una jerarquía de tipos, en la cual el tipo `Any` es el "padre" de todos los tipos, podemos verficar si un tipo es un subtipo de otro mediante el operador `<:` el cuál puede ser empleado como

In [3]:
Float64 <: Float64 # Un tipo siempre es subtipo de sí mismo

true

In [4]:
Float64 <: AbstractFloat # AbstractFloat es un tipo que es padre de todos los Float

true

In [5]:
Float32 <: AbstractFloat

true

In [6]:
AbstractFloat <: Any

true

Es decir `TipoHijo <: TipoPadre` se lee: ¿`TipoHijo` es subtipo de `TipoPadre`?

## Tipos Compuestos

Una de las cosas interesantes de Julia es la capacidad de definir tipos compuestos, un `tipo compuesto` puede ser pensado como una estructura capaz de alojar dentro de ella diversos campos que pueden contener esencialmente cualquier objeto. En Julia existen dos clases de tipos compuestos: inmutables y mutables.

Los tipos inmutables se caracterizan por la propiedad de que al ser inicializados estos no pueden son modificados, contrario a lo que sucede con los mutables.

Podemos definir nuestros propios tipos inmutables de la siguiente manera:

In [7]:
struct TipoInmutable
    campo1
    campo2
end

Para un tipo mutable:

In [8]:
mutable struct TipoMutable
    campo1
    campo2
end

**Ojo:** por cuestiones internas a Julia una vez dada la definición de un tipo, éste no puede ser reedefinido, por lo que si queremos actualizar la definición de alguno de nuestros tipos será necesario reiniciar el kernel. 

Una vez dadas las definiciones de nuestro tipo podemos inicializarlo llamando al nombre del tipo y entre paréntesis los valores de los campos con los que fue definido

In [9]:
TipoInmutable(1, 2)

TipoInmutable(1, 2)

In [10]:
TipoInmutable("Hola", "Mundo")

TipoInmutable("Hola", "Mundo")

Y podemos asignar esto a una variable de la manera usual

In [11]:
tipo_inmutable = TipoInmutable("Soy ", "inmutable")
tipo_mutable = TipoMutable("Soy ", "mutable");

Podemos accesar a los campos del tipo mediante la sintáxis `tipo.campo`

In [12]:
tipo_inmutable.campo1 * tipo_inmutable.campo2

"Soy inmutable"

In [13]:
tipo_mutable.campo1 * tipo_mutable.campo2

"Soy mutable"

Mediante esto es posible modificar los valores de los campos(para el caso de tipo mutables)

In [14]:
tipo_mutable.campo1 = "Me "; tipo_mutable.campo2 = "han mutado!"
tipo_mutable.campo1 * tipo_mutable.campo2

"Me han mutado!"

Notemos que esto no es posible para el un tipo inmutable

In [15]:
tipo_inmutable.campo1 = " "

ErrorException: type TipoInmutable is immutable

**Ojo:** En el caso de los tipos mutables, aunque dos objetos tengan los mismos valores en sus campos, **éstos no son iguales**, lo cual podría causar algunas confusiones, veamos

In [16]:
objeto1 = TipoMutable(1, 1)

TipoMutable(1, 1)

In [17]:
objeto2 = TipoMutable(1, 1)

TipoMutable(1, 1)

In [18]:
objeto1 == objeto2

false

Además notemos 

In [19]:
objeto2 = objeto1

TipoMutable(1, 1)

In [20]:
objeto1 == objeto2

true

En este caso sí se trata del mismo objeto, es decir las variables objeto1 y objeto2 "apuntan" hacía el mismo lugar, por lo que la modificación de uno de sus campos afecta a ambas variables

In [21]:
objeto1.campo1 = 0
objeto1

TipoMutable(0, 1)

In [22]:
objeto2

TipoMutable(0, 1)

Este comportamiento es el que se presenta con los arreglos puesto que son objetos mutables.

En ocasiones podemos no conocer los nombres de los campos de un tipo dado, es posible ver cuáles son mediante la función `fieldnames`

In [23]:
fieldnames(TipoInmutable)

(:campo1, :campo2)

In [24]:
fieldnames(TipoMutable)

(:campo1, :campo2)

Para conocer si un objeto dado es de un cierto tipo podemos emplear `isa`

In [25]:
isa(tipo_inmutable, TipoInmutable)

true

In [26]:
isa(tipo_inmutable, TipoMutable)

false

Hay situaciones en las que necesitamos que los campos de nuestro tipo estén restringidos a ser de cierta forma, en las definiciones que se han dado anteriormente los campos pueden ser cualquier cosa (incluso otro tipo!)

In [27]:
obj = TipoInmutable(1, x -> x^2)

TipoInmutable(1, getfield(Main, Symbol("##3#4"))())

In [28]:
obj.campo2(4)

16

In [29]:
TipoInmutable(TipoMutable(0, 0), TipoInmutable(1, 1))

TipoInmutable(TipoMutable(0, 0), TipoInmutable(1, 1))

Por ejemplo, tal vez necesitamos que nuestro tipo tenga campos cuyos únicos valores sean números, podemos lograr esto mediante la siguiente definición:

In [30]:
struct TipoNumero
    campo1::Number
    campo2::Number
end

De esta forma los únicos valores permitidos serán números, y en caso de querer construir un objeto que no satisface esta definición obtenemos un error

In [31]:
TipoNumero(1, 1)

TipoNumero(1, 1)

In [32]:
TipoNumero("Hola", "mundo")

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Number
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

En algunos casos es útil poder especificar los campos como la unión de dos tipos distintos, esto se consigue mediante `Union`

In [33]:
struct EnteroFlotante64
    campo1::Union{Float64, Int64}
end

In [34]:
EnteroFlotante64(42)

EnteroFlotante64(42)

In [35]:
EnteroFlotante64(42.5)

EnteroFlotante64(42.5)

In [36]:
EnteroFlotante64("s")

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Union{Float64, Int64}
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

## Tipos Paramétricos

Un tipo puede estar definido mediante un parámetro, esto son los llamados tipos paramétricos, la definición de éstos es muy similar a lo visto anteriormente

In [37]:
struct TipoParam{T}
    campo1::T
    campo2::T
end

Es decir, un tipo así definido implica una definición de `TipoParam` para cada posible valor de `T`, notemos que ambos campos han de pertenecer al mismo tipo, al crear el objeto Julia sabe determinar quién es `T`

In [38]:
TipoParam(1, 1)

TipoParam{Int64}(1, 1)

In [39]:
TipoParam(1., 1.)

TipoParam{Float64}(1.0, 1.0)

In [40]:
TipoParam("Soy", "paramétrico")

TipoParam{String}("Soy", "paramétrico")

Si se intenta construir un objeto con campos de distinto tipo se obtendrá un error

In [41]:
TipoParam(1, 1.)

MethodError: MethodError: no method matching TipoParam(::Int64, ::Float64)
Closest candidates are:
  TipoParam(::T, !Matched::T) where T at In[37]:2

Esto se puede solucionar mediante la introducción de más parámetros en la definición del tipo

In [42]:
struct Tipo2Params{S, T}
    campo1::S
    campo2::T
end

En este caso se está creando la definición del tipo para cada posible combinación de `S`, `T`

In [43]:
Tipo2Params(1, 2.)

Tipo2Params{Int64,Float64}(1, 2.0)

In [44]:
Tipo2Params(1, "s")

Tipo2Params{Int64,String}(1, "s")

Los tipos paramétricos pueden ser restringidos a un subconjunto de todos los tipos disponible en Julia, para esto se hace uso de la notación `T <: Tipo` donde `T` es el parámetro, este enunciado se lee: `T` es un subtipo de `Tipo`, a diferencia de lo visto en la primer parte de este notebook, en este contexto esta sintáxis representa una afirmación.

In [45]:
struct ParamRestr{T <: Real}
    campo::T
end

In [46]:
ParamRestr(2.)

ParamRestr{Float64}(2.0)

In [47]:
ParamRestr(2 + 2im)

MethodError: MethodError: no method matching ParamRestr(::Complex{Int64})
Closest candidates are:
  ParamRestr(!Matched::T<:Real) where T<:Real at In[45]:2

Los tipos paramétricos son una manera conveniente de definir objetos de cualquier clase obteniendo un mejor rendimento que empleando definiciones sin especificar el tipo de los campos(como las primeras definiciones que se vieron al principio de este notebook), esto se debe a que el compilador de Julia es capaz de determinar los valores que tendrán los campos del tipo con sólo ver la definición de éste, puede leerse más acerca de esto en la sección de ["Performance Tips"](https://docs.julialang.org/en/v1/manual/performance-tips/#Type-declarations-1) de la documentación de Julia.

Veamos un ejemplo de esto [1]

In [48]:
struct Cubo
    largo
    ancho
    alto
end
volumen(c::Cubo) = c.largo * c.ancho * c.alto

struct Cubo2
    largo::Float64
    ancho::Float64
    alto::Float64
end
volumen(c::Cubo2) = c.largo * c.ancho * c.alto

struct Cubo3{T <: Real}
    largo::T
    ancho::T
    alto::T
end

volumen(c::Cubo3) = c.largo * c.ancho * c.alto

volumen (generic function with 3 methods)

In [49]:
using BenchmarkTools

In [50]:
c1 = Cubo(2.1, 2.2, 2.3)
c2 = Cubo2(2.1, 2.2, 2.3)
c3 = Cubo3(2.1, 2.2, 2.3)

Cubo3{Float64}(2.1, 2.2, 2.3)

In [51]:
volumen(c1) == volumen(c2) == volumen(c3)

true

In [52]:
@btime volumen(c1)

  31.586 ns (1 allocation: 16 bytes)


10.626000000000001

In [53]:
@btime volumen(c2)

  10.475 ns (1 allocation: 16 bytes)


10.626000000000001

In [54]:
@btime volumen(c3)

  29.106 ns (1 allocation: 16 bytes)


10.626000000000001

En general, ser más específico respecto a los valores que pueden tomar los campos de nuestros tipos se traduce en que el compilador de Julia pueda generar código más eficiente.

## Métodos

En Julia para una función dada pueden existir diversas definiciones(métodos) las cuales se aplican dependiendo del tipo de argumentos con las que la función es llamada, a esto se le conoce como *multiple dispatch*, esto permite definir comportamientos distintos de nuestra función para diversos casos, ya hemos visto ejemplos de esto, por ejemplo con operaciones elementales

In [55]:
(1 + 1)

2

In [56]:
[1, 1] + [1, 1]

2-element Array{Int64,1}:
 2
 2

In [57]:
1 * 1

1

In [58]:
[1 1] * [1, 1]

1-element Array{Int64,1}:
 2

In [59]:
"Hola " * "mundo"

"Hola mundo"

Podemos ver cuáles son los métodos de una función utilizando la función `methods`

In [60]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:92
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:93
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:449
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:455
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:325
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:333
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:355
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:353
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:391
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:78
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:76
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:59
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:32
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:20
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:24
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:82
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:52
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:28
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:80
[43]

Para definir métodos de una función basta definir la función pero con distinto número de parámetros o distintos tipos, al usar la función Julia sabrá cuál definición emplear.

In [61]:
mi_func(x, y) = "Dos parámetros"
mi_func(x) = "Un parámetro"
mi_func(x::Float64, y::Float64) = "Dos flotantes"

mi_func (generic function with 3 methods)

In [62]:
mi_func(1, 2.)

"Dos parámetros"

In [63]:
mi_func(" ")

"Un parámetro"

In [64]:
mi_func(1, 2.)

"Dos parámetros"

Al igual que en la construcción de tipos es posible parametrizar una función, esto se hace de la manera siguiente:

In [65]:
mi_func(x::T, y::T) where {T} = "Los parámetros son del mismo tipo"

mi_func (generic function with 4 methods)

In [66]:
mi_func("s", "t")

"Los parámetros son del mismo tipo"

In [67]:
mi_func(x -> x^2, x -> x^2)

"Dos parámetros"

In [68]:
mi_func(1, 1)

"Los parámetros son del mismo tipo"

Notemos como la definición más específica es la que se escoge.

In [69]:
mi_func(1., 1.)

"Dos flotantes"

En general, parametrizar funciones de esta manera sólo se recomienda cuándo necesitemos hacer uso del parámetro `T` dentro de nuestra función, siempre será más útil dar explícitamente el tipo de los argumentos de la función mediante la sintáxis

```julia
function mi_func(x::TipoA, y::TipoB)
    ...
end
```
Tal como se ha hecho en uno de los métodos para `mi_func`, en dónde se ha especificado que sus argumentos son `Float64`.

En varias aplicaciones será útil poder escribir métodos de funciones presentes en Julia base para nuestros propios tipos, para poder hacer esto es necesario importar las funciones que queremos "extender" y posteriormente escribir el método correspondiente a nuestro tipo, veamos un ejemplo de esto creando nuestro priopio número complejo

In [70]:
import Base: +, -, *

In [71]:
struct Complejo{S, T}
    a::S
    b::T
end

In [72]:
+(c1::Complejo, c2::Complejo) = Complejo(c1.a + c2.a, c1.b + c2.b)
-(c1::Complejo, c2::Complejo) = Complejo(c1.a - c2.a, c1.b - c2.b)
*(c1::Complejo, c2::Complejo) = Complejo((c1.a * c2.a) - (c1.b * c2.b), (c1.a * c2.b) + (c1.b * c2.a)) 

* (generic function with 343 methods)

In [73]:
c1 = Complejo(1., 2)

Complejo{Float64,Int64}(1.0, 2)

In [74]:
c2 = Complejo(2, 2.)

Complejo{Int64,Float64}(2, 2.0)

In [75]:
c1 + c2

Complejo{Float64,Float64}(3.0, 4.0)

In [76]:
c1 - c2

Complejo{Float64,Float64}(-1.0, 0.0)

In [77]:
c1 * c2

Complejo{Float64,Float64}(-2.0, 6.0)

Muchas veces quisieramos que el despliegue de nuestros tipos sea distinto al que Julia escoge por default, como en el caso anterior, sería agradable que Julia desplegara la clásica notación de número complejo $a + ib$ cada vez que empleamos un complejo, esto puede hacerse agregando un método para nuestro tipo `Complejo`a la función `show` que es la que se encarga de imprimir cosas

In [78]:
TipoInmutable(1, 2)

TipoInmutable(1, 2)

In [79]:
import Base: show

function show(io::IO, c::Complejo)
    print("$(c.a) + $(c.b)i")
end

show (generic function with 352 methods)

In [80]:
c1

1.0 + 2i

Por último, veamos un tipo de funciones de especial interés en la creación de objetos: los constructores.

Un constructor es una función encargada de crear objetos de un tipo dado, la función constructor se define con el mismo nombre del tipo del cual será constructor(los constructores son individuales para cada tipo), por default Julia crea una función constructor que lo único que hace es pasar los argumentos que recibe como valores para los campos del tipo en el orden dado, es decir, si tenemos un tipo `MiTipo` con campos `campo1`, `campo2`, la llamada al constructor

```julia
mi_tipo = MiTipo(1, 2)
```
creará un objeto tal que `mi_tipo.campo1 = 1, mi_tipo.campo2 = 2`.





No siempre deseamos este comportamiento, para eso es posible definir nuestros propios constructores, por ejemplo

In [81]:
struct Punto2D{T}
    x::T
    y::T
end

# Notemos que pueden existir diversos métodos para el constructor
Punto2D() = Punto2D(rand(), rand())
Punto2D(x) = Punto2D(x, x)

Punto2D

Nuestros constructores siempre recaerán en el constructor por default, como lo vemos en lo anterior, `Punto2D` a la derecha de la igualdad está haciendo referencia al constructor por default.

El primer método que hemos definido crea un `Punto2D` con coordenadas aleatorias si mandamos llamar al constructor sin argumentos, el segundo crea un punto con la misma coordenada si mandamos llamar al constructor con un solo argumento, veamos

In [82]:
Punto2D()

Punto2D{Float64}(0.2910204121739257, 0.37630278225013103)

In [83]:
Punto2D(2)

Punto2D{Int64}(2, 2)

## Árbol Binario con tipos

In [84]:
import Base: maximum, minimum, length, push!, show

# Primero creamos un nodo del árbol, éste tendrá como campos los nodos(si existen): padre, izq(hijo), der(hijo),
# y el valor 

mutable struct Nodo{T}
    padre::Union{Nodo, Nothing}
    izq::Union{Nodo, Nothing}
    der::Union{Nodo, Nothing}
    valor::T
end

# Este constructor crea un nodo sin padre ni hijos cuando se manda llamar sólo con el valor
Nodo(valor::T) where {T} = Nodo{T}(nothing, nothing, nothing, valor) 

# La estructura árbol almacenará solamente dos cosas, el tamaño del árbol(esto es cuántos nodos tiene) y un nodo raíz
# se podrá acceder a los demás nodos siguiendo los campos izq, y der
mutable struct Arbol
    raiz::Union{Nodo, Nothing}
    tamaño::Int
end

# Crea un árbol vacío
Arbol() = Arbol(nothing, 0)

length(A::Arbol) = A.tamaño

# Extiendo show para una mejor visalización de un nodo
# al unir varios nodos será imposible visualizar correctamnte el nodo(vea qué pasa si no se extiende)
# es por eso que es necesario extender esta función
function show(io::IO, n::Nodo)
    print("valor: $(n.valor)")
end

show (generic function with 353 methods)

Para ilustrar el funcionamiento de estas estructuras intentemos construir a "mano" el siguiente árbol binario

![](./arbol.png)

In [85]:
# Primero creamos un nodo que será la raíz, cuyo valor es 27
# De la definición del constructor que hemos dado, los campos padre, der, izq, serán nothing
raiz = Nodo(27)

valor: 27

Creemos los nodos para los valores del árbol

In [86]:
vals = [14, 35, 10, 19, 31, 42]
nodos = [Nodo(v) for v in vals]; # Esto crea un arreglo de estructuras nodo

Hay que unir los nodos siguiendo el ordenamiento del árbol binario 

In [87]:
raiz.izq = nodos[1] # como 14 es menor que 27, entonces va a la izq.

valor: 14

In [88]:
# Como al inicializar los nodos, éstos tienen campos nothing excepto por el campo valor
# es necesario asignar el padre a este nodo
nodos[1].padre = raiz

valor: 27

Repetimos este procedimiento para cada nodo en `nodos`

In [89]:
raiz.der = nodos[2]
nodos[2].padre = raiz

raiz.izq.izq = nodos[3]
nodos[3].padre = raiz.izq

raiz.izq.der = nodos[4]
nodos[4].padre = raiz.izq

raiz.der.izq = nodos[5]
nodos[5].padre = raiz.der

raiz.der.der = nodos[6]
nodos[6].padre = raiz.der

valor: 35

Con esto hemos creado el árbol binario, y para acceder a cada uno de los nodos del árbol basta ir descendiendo por los campos `izq` y `der` del nodo `raiz`.

In [90]:
raiz.der.valor

35

In [91]:
raiz.izq.valor

14

Ahora simplemente creamos un objeto tipo `Arbol` cuya raíz será justamente lo anterior y tendrá como longitud el número de nodos que hemos creado anteriormente

In [92]:
arbol = Arbol(raiz, length(nodos))

Arbol(, 6)

valor: 27

In [93]:
length(arbol)

6

In [94]:
function push!(A::Arbol, valor)
    # Primero se crea un nodo con el valor que se desea introducir al árbol
    nodo = Nodo(valor)
    y = nothing # Esto servirá para dar seguimiento a qué nodos hemos visitado
    x = A.raiz 
    while !isa(x, Nothing)
        y = x
        nodo.valor < x.valor ? x = x.izq : x = x.der
    end
    nodo.padre = y
    if isa(y, Nothing)
        A.raiz = nodo
    elseif nodo.valor < y.valor
        y.izq = nodo
    else
        y.der = nodo
    end
    A.tamaño += 1
end 

push! (generic function with 25 methods)

In [95]:
function minimum(n::Nodo)
    while !isa(n.izq, Nothing)
        n = n.izq
    end
    n
end

function maximum(n::Nodo)
    while !isa(n.der, Nothing)
        n = n.der
    end
    n
end

minimum(A::Arbol) = minimum(A.raiz)
maximum(A::Arbol) = maximum(A.raiz)

function buscar(A::Arbol, valor)
    x = A.raiz
    while !isa(x, Nothing) && (valor != x.valor)
        valor < x.valor ? x = x.izq : x = x.der
    end
    x
end

buscar (generic function with 1 method)

In [96]:
A = Arbol()

nums = rand(10)

for n in nums push!(A, n) end

sort(nums)

10-element Array{Float64,1}:
 0.0027591631691152863
 0.01013350381856859  
 0.041312680639209676 
 0.14886403619903188  
 0.22316720760866748  
 0.4407491576421012   
 0.4858354033772412   
 0.5270821161695043   
 0.6043248127728196   
 0.9196296849990342   

In [97]:
minimum(A)

valor: 0.0027591631691152863

In [98]:
maximum(A)

valor: 0.9196296849990342

In [99]:
buscar(A, nums[rand(1:length(nums))])

valor: 0.22316720760866748

[1] Ejemplo tomado de https://www.cs.purdue.edu/homes/hnassar/JPUG/performance.html 